<a href="https://colab.research.google.com/github/Satwikram/NLP-Using-Deep-Learning/blob/master/Fake%20News%20Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Author: Satwik Ram K

### Connecting Kaggale

In [1]:
from google.colab import files

files.upload()


! mkdir ~/.kaggle


! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [2]:
! kaggle datasets download -d jruvika/fake-news-detection

fake-news-detection.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
!unzip /content/fake-news-detection.zip

Archive:  /content/fake-news-detection.zip
replace data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace data.h5? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


### Importing Dependencies

In [1]:
import numpy as np
import pandas as pd
import re
import tensorflow as tf
from tensorflow import keras
import math 
from bs4 import BeautifulSoup

### Loading Dataset

In [2]:
dataset = pd.read_csv('/content/data.csv')

In [3]:
dataset.head()

,URLs,Headline,Body,Label
0,http://www.bbc.com/news/world-us-canada-414191...,Four ways Bob Corker skewered Donald Trump,Image copyright Getty Images\nOn Sunday mornin...,1
1,https://www.reuters.com/article/us-filmfestiva...,Linklater's war veteran comedy speaks to moder...,"LONDON (Reuters) - “Last Flag Flying”, a comed...",1
2,https://www.nytimes.com/2017/10/09/us/politics...,Trump’s Fight With Corker Jeopardizes His Legi...,The feud broke into public view last week when...,1
3,https://www.reuters.com/article/us-mexico-oil-...,Egypt's Cheiron wins tie-up with Pemex for Mex...,MEXICO CITY (Reuters) - Egypt’s Cheiron Holdin...,1
4,http://www.cnn.com/videos/cnnmoney/2017/10/08/...,Jason Aldean opens 'SNL' with Vegas tribute,"Country singer Jason Aldean, who was performin...",1


In [4]:
print(dataset['Label'][0])
dataset['Body'][0]


1


'Image copyright Getty Images\nOn Sunday morning, Donald Trump went off on a Twitter tirade against a member of his own party.\nThis, in itself, isn\'t exactly huge news. It\'s far from the first time the president has turned his rhetorical cannons on his own ranks.\nThis time, however, his attacks were particularly biting and personal. He essentially called Tennessee Senator Bob Corker, the chair of the powerful Senate Foreign Relations Committee, a coward for not running for re-election.\nHe said Mr Corker "begged" for the president\'s endorsement, which he refused to give. He wrongly claimed that Mr Corker\'s support of the Iranian nuclear agreement was his only political accomplishment.\nUnlike some of his colleagues, Mr Corker - free from having to worry about his immediate political future - didn\'t hold his tongue.\nSkip Twitter post by @SenBobCorker It\'s a shame the White House has become an adult day care center. Someone obviously missed their shift this morning. — Senator Bo

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   URLs      4009 non-null   object
 1   Headline  4009 non-null   object
 2   Body      3988 non-null   object
 3   Label     4009 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 125.4+ KB


In [6]:
len(list(dataset))

4

### Checking for NaN values

In [7]:
dataset.isna().sum()

URLs         0
Headline     0
Body        21
Label        0
dtype: int64

In [8]:
dataset['Label'].unique()

array([1, 0])

### Dropping Unwanted Information

In [9]:
dataset.drop(columns = ['URLs', 'Headline'], axis = 1, inplace = True)

In [10]:
dataset.head(0)

,Body,Label


### Dropping NaN Values

In [11]:
dataset.isna().sum()

Body     21
Label     0
dtype: int64

In [12]:
dataset.dropna(inplace = True)

In [13]:
dataset.isna().sum()

Body     0
Label    0
dtype: int64

In [14]:
len(dataset)

3988

### Taking X and Y

In [15]:
X = dataset['Body'].values
y = dataset['Label'].values

In [16]:
print(len(X))
type(X)

3988


numpy.ndarray

In [17]:
X[200]

"Garcia has now won five titles on the WTA tour\nFrance's Caroline Garcia beat new world number one Simona Halep to win the China Open and overtake Briton Johanna Konta in the race for Singapore.\nUnseeded Garcia edged Romanian Halep 6-4 7-6 (7-3) for her second successive title after victory at the Wuhan Open.\nShe hit 17 winners and saved nine break points in the second set to move into the eighth and final qualifying spot for the tour finals.\nKonta must now reach the final in Moscow to have a chance of overtaking.\nBritain's Konta will play in the Kremlin Cup, which begins on 16 October, but Garcia has put herself in pole position with a hard-fought victory over Halep.\nHalep moved to the top of the world rankings following her semi-final victory over Jelena Ostapenko on Saturday, and broke Garcia in the first game of the final.\nBut Garcia immediately broke back, before battling to take a tight first set.\nHalep then wasted countless opportunities in the second before her final sh

### Cleaning the news body using Regular Expression


In [18]:
def clean_news(body):
   body = BeautifulSoup(body, 'lxml').get_text()
   body = re.sub(r"@[A-Za-z0-9]+", "", body)
   body = re.sub(r"https?://[A-Za-z0-9./]", "", body)
   body = re.sub(r"[^A-Za-z]", "", body)
   body = re.sub(r" +", '', body)
   return body

In [19]:
clean_news(X[3000])

'WarningSomethingBigIsAbouttoHappeninAmericatheMartialLawIsComingChanceitWillHappenRedListandBlueList'

In [20]:
for i in range(len(X)):
  X[i] = clean_news(X[i])


In [21]:
X[3000]

'WarningSomethingBigIsAbouttoHappeninAmericatheMartialLawIsComingChanceitWillHappenRedListandBlueList'

### Tokenization

In [22]:
import tensorflow_datasets as tfds
tfds.__version__

'2.1.0'

In [23]:
from tqdm import tqdm

In [24]:
#tokenizer = tfds.features.text.SubwordTextEncoder.build_from_corpus(X,target_vocab_size = 2**12)

In [25]:
2**12

4096

### Saving the Tokenizer

In [26]:
#tokenizer.save_to_file('tokenizer')

### Load the saved tokenizer

In [27]:
encoder = tfds.features.text.SubwordTextEncoder.load_from_file('/content/tokenizer')

### Encode into Numbers

In [28]:
X = [encoder.encode(sentence) for sentence in X]


### Padding

In [29]:
MAX_LEN = max([len(sentence) for sentence in X])

In [30]:
MAX_LEN

7025

In [31]:
X = tf.keras.preprocessing.sequence.pad_sequences(
    X, value = 0, padding = 'post', maxlen = MAX_LEN 
)

In [32]:
print(X[100])
print("The sentiment is:", y[100])


[3671 7584 7598 ...    0    0    0]
The sentiment is: 1


In [33]:
int(0.25*(len(X)))

997

### Splitting data into Train and Test

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 388)

### Config

In [35]:
VOCAB_SIZE = encoder.vocab_size
print(VOCAB_SIZE)


7765


### Building the model

In [36]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(VOCAB_SIZE, 128))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences = True)))
model.add(tf.keras.layers.Dropout(0.5))
#model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences = True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)))
model.add(tf.keras.layers.Dense(128, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))

In [37]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         993920    
_________________________________________________________________
bidirectional (Bidirectional (None, None, 256)         263168    
_________________________________________________________________
dropout (Dropout)            (None, None, 256)         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               394240    
_________________________________________________________________
dense (Dense)                (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1

### Removing the Temporary Logs

In [38]:
!rm -r /tmp/logs

rm: cannot remove '/tmp/logs': No such file or directory


### Creating Checkpoints and Callbacks

In [39]:
import os
import datetime
logdir = os.path.join("/tmp/logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq = 1)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath = '/tmp/fake_news_predictor.hdf5', verbose = 1, save_best_only = True)


### Compiling the model

In [40]:
model.compile(optimizer = 'adam',
              loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
              metrics = ['accuracy'])

### Training the Data

In [ ]:
history = model.fit(X_train, y_train, epochs = 10, batch_size = 32, validation_split = 0.1, callbacks = [checkpoint, tensorboard_callback])

Epoch 1/10
 1/85 [..............................] - ETA: 0s - loss: 0.7081 - accuracy: 0.5625WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
85/85 [==============================] - ETA: 0s - loss: 0.6963 - accuracy: 0.5318 
Epoch 00001: val_loss improved from inf to 0.69315, saving model to /tmp/fake_news_predictor.hdf5
85/85 [==============================] - 5134s 60s/step - loss: 0.6963 - accuracy: 0.5318 - val_loss: 0.6931 - val_accuracy: 0.5033
Epoch 2/10
85/85 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.5307  
Epoch 00002: val_loss did not improve from 0.69315
85/85 [==============================] - 5433s 64s/step - loss: 0.6931 - accuracy: 0.5307 - val_loss: 0.6931 - val_accuracy: 0.5033
Epoch 3/10
85/85 [==================

In [ ]:
#model.save('/tmp/final_model.hdf5')